## Using LangChain tool in AutoGen

In [1]:
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
# AutoGen's wrapper:
from autogen_ext.tools.langchain import LangChainToolAdapter

# LangChain tools:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_core.tools import Tool

prompt = """Your task is to find a one-way non-stop flight from MAA to DXB in Feb 2026.
First search online for promising deals.
Next, write all the deals to a file called flights.md with full details.
Finally, select the one you think is best and reply with a short summary.
Reply with the selected flight only, and only after you have written the details to the file."""

serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name="internet_search", func=serper.run, description="useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)
autogen_tools = [autogen_serper]

langchain_file_management_tools = FileManagementToolkit(root_dir="sandbox").get_tools()
for tool in langchain_file_management_tools:
    autogen_tools.append(LangChainToolAdapter(tool))

for tool in autogen_tools:
    print(tool.name, tool.description)

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
agent = AssistantAgent(name="searcher", model_client=model_client, tools=autogen_tools, reflect_on_tool_use=True)
message = TextMessage(content=prompt, source="user")
result = await agent.on_messages([message], cancellation_token=CancellationToken())
for innermessage in result.inner_messages:
    print(innermessage.content)
display(Markdown(result.chat_message.content))

internet_search useful for when you need to search the internet
copy_file Create a copy of a file in a specified location
file_delete Delete a file
file_search Recursively search for files in a subdirectory that match the regex pattern
move_file Move or rename a file from one location to another
read_file Read file from disk
write_file Write file to disk
list_directory List files and directories in a specified folder
[FunctionCall(id='call_nNwgENDz92olttevCUzEgLur', arguments='{"query":"one-way non-stop flights from MAA to DXB in February 2026"}', name='internet_search')]
[FunctionExecutionResult(content='There are 30 flights per week flying from Chennai to Dubai, as of February 2026. ... The cheapest price we\'ve seen for a one-way Chennai Airport to Dubai Airport ... Direct flights from Chennai to Dubai, 1 *daily frequency varies ; Chennai to Dubai flight duration, 4 hours and 35 minutes (for direct flights) ; Chennai to Dubai ... Cheap Flights from Chennai (MAA) to Dubai (DXB) start

I found some promising deals for one-way non-stop flights from Chennai (MAA) to Dubai (DXB) in February 2026. Here are the details I will write to the file `flights.md`:

1. **Airline:** Oman Air
   - **Departure:** 02:50 local time from Chennai (MAA)
   - **Arrival:** 05:25 local time in Dubai (DXB)
   - **Duration:** 4 hours and 35 minutes
   - **Price:** $124

2. **Airline:** IndiGo
   - **Departure:** 21:10 local time from Chennai (MAA)
   - **Arrival:** 23:45 local time in Dubai (DXB)
   - **Duration:** 4 hours and 5 minutes
   - **Price:** $238

3. **Airline:** Air India
   - **Departure:** 15:00 local time from Chennai (MAA)
   - **Arrival:** 17:35 local time in Dubai (DXB)
   - **Duration:** 4 hours and 35 minutes
   - **Price:** $210

Next, I will write this information to the `flights.md` file.

In [3]:
message = TextMessage(content="Ok Proceed", source="user")
result = await agent.on_messages([message], cancellation_token=CancellationToken())
for innermessage in result.inner_messages:
    print(innermessage.content)
display(Markdown(result.chat_message.content))

[FunctionCall(id='call_MOF1k31MKFvpy3QDyVnnjQtr', arguments='{"file_path":"flights.md","text":"### One-way Non-stop Flights from MAA to DXB in February 2026\\n\\n1. **Airline:** Oman Air\\n   - **Departure:** 02:50 local time from Chennai (MAA)\\n   - **Arrival:** 05:25 local time in Dubai (DXB)\\n   - **Duration:** 4 hours and 35 minutes\\n   - **Price:** $124\\n\\n2. **Airline:** IndiGo\\n   - **Departure:** 21:10 local time from Chennai (MAA)\\n   - **Arrival:** 23:45 local time in Dubai (DXB)\\n   - **Duration:** 4 hours and 5 minutes\\n   - **Price:** $238\\n\\n3. **Airline:** Air India\\n   - **Departure:** 15:00 local time from Chennai (MAA)\\n   - **Arrival:** 17:35 local time in Dubai (DXB)\\n   - **Duration:** 4 hours and 35 minutes\\n   - **Price:** $210\\n","append":false}', name='write_file')]
[FunctionExecutionResult(content='File written successfully to flights.md.', name='write_file', call_id='call_MOF1k31MKFvpy3QDyVnnjQtr', is_error=False)]


I have written the details of the flights to the `flights.md` file successfully. 

Among the options, I selected the flight from **Oman Air** as the best choice:

- **Departure:** 02:50 local time from Chennai (MAA)
- **Arrival:** 05:25 local time in Dubai (DXB)
- **Duration:** 4 hours and 35 minutes
- **Price:** $124

This flight offers the lowest fare with a reasonable duration for a non-stop journey. 

TERMINATE

## Team Interactions

In [4]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name="internet_search", func=serper.run, description="useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

prompt = "find a one-way non-stop flight from MAA to DXB in February 8, 2026"

primary_agent = AssistantAgent(
    name="primary",
    model_client=model_client,
    tools=[autogen_serper],
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Incorporate any feedback you receive."
)

evaluation_agent = AssistantAgent(
    name="evaluator",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedback is addressed. APPROVE by default when you gave less than or equal to 3 feedback"
)

text_termination = TextMentionTermination("APPROVE")

team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination)

In [5]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f"{message.source}:\n{message.content}\n\n")

CancelledError: 

## Introducing MCP!

Our first look at the Model Context Protocol from Anthropic -

Autogen makes it easy to use MCP tools, just like LangChain tools.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Get the fetch tool from mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"], read_timeout_seconds=30)
fetcher = await mcp_server_tools(fetch_mcp_server)

# Create an agent that can use the fetch tool.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=fetcher, reflect_on_tool_use=True)  # type: ignore

# Let the agent fetch the content of a URL and summarize it.
result = await agent.run(task="Review edwarddonner.com and summarize what you learn. Reply in Markdown.")
display(Markdown(result.messages[-1].content))